In [1]:
%load_ext autoreload
%autoreload 2

import sys
sys.path.append('/home/jupyter/checkmate-histo/consolidated_workflow/immunoprofile_analysis/')

from imports import *
import xarray
from sklearn.neighbors import KNeighborsRegressor
from sklearn.neighbors import KNeighborsClassifier

from tqdm import tqdm_notebook
set_rc()

In [2]:
import networkx as nx
from scipy.stats import pearsonr

def prune_self_interactions(adata, cell_col = 'cell_type'):
    g = nx.Graph(adata.obsp['spatial_connectivities'])

    print('pre-prune (E,V): ', len(g.edges), len(g.nodes))

    full_edges = np.array(g.edges)
    full_edges_df = pd.DataFrame(full_edges, columns=['a','b'])

    ct_df = adata.obs[[cell_col, 'detailed_cell_type']]
    i = ct_df.iloc[full_edges[:,0]][cell_col]
    j = ct_df.iloc[full_edges[:,1]][cell_col]
    full_edges_df['ct_a'] = i.values 
    full_edges_df['ct_b'] = j.values
    
    i = ct_df.iloc[full_edges[:,0]]['detailed_cell_type']
    j = ct_df.iloc[full_edges[:,1]]['detailed_cell_type']
    full_edges_df['dct_a'] = i.values 
    full_edges_df['dct_b'] = j.values

    filtered_full_edges_df = full_edges_df.loc[full_edges_df['ct_a'] != full_edges_df['ct_b']]

    g = nx.Graph()
    g.add_nodes_from(list(range(len(adata.obs))))
    g.add_edges_from(filtered_full_edges_df[['a','b']].values)

    print('post-prune (E,V): ', len(g.edges), len(g.nodes))

    adata.obsp['spatial_connectivities'] = nx.to_scipy_sparse_matrix(g)

    degrees = [d for (n,d) in g.degree()]
    adata.obs['degree'] = degrees

    filtered_full_edges_df[['tumor_node', 'cd8_node']] = np.stack(filtered_full_edges_df.apply(lambda x: fetch_subexpression(x), 1))

    filtered_full_edges_df['res_pdl1_in_tumor'] = adata.obs.loc[filtered_full_edges_df['tumor_node'].astype(str), 'residual_pdl1_cond_autofluorescence'].values
    filtered_full_edges_df['res_pd1_in_cd8'] = adata.obs.loc[filtered_full_edges_df['cd8_node'].astype(str), 'residual_pd1_cond_autofluorescence'].values

    filtered_full_edges_df['tumor_degree'] = adata.obs.loc[filtered_full_edges_df['tumor_node'].astype(str), 'degree'].values
    filtered_full_edges_df['cd8_degree'] = adata.obs.loc[filtered_full_edges_df['cd8_node'].astype(str), 'degree'].values

    return filtered_full_edges_df

def fetch_subexpression(row):
    if row['ct_a'] == 'Tumor+':
        tumor_idx = row['a']
        cd8_idx = row['b']

    else:
        tumor_idx = row['b']
        cd8_idx = row['a']
    return tumor_idx, cd8_idx

def fetch_full_edge_df(adata, spatial_key = 'unpruned_delaunay_connectivities', cell_col = 'cell_type'):
    g = nx.Graph(adata.obsp[spatial_key])

    print('pre-prune (E,V): ', len(g.edges), len(g.nodes))

    full_edges = np.array(g.edges)
    full_edges_df = pd.DataFrame(full_edges, columns=['a','b'])

    ct_df = adata.obs[[cell_col, 'detailed_cell_type']]
    i = ct_df.iloc[full_edges[:,0]][cell_col]
    j = ct_df.iloc[full_edges[:,1]][cell_col]
    full_edges_df['ct_a'] = i.values 
    full_edges_df['ct_b'] = j.values
    
    i = ct_df.iloc[full_edges[:,0]]['detailed_cell_type']
    j = ct_df.iloc[full_edges[:,1]]['detailed_cell_type']
    full_edges_df['dct_a'] = i.values 
    full_edges_df['dct_b'] = j.values
    
    return full_edges_df

def add_degree_centrality(adata, connectivity_key):
    g = nx.Graph(adata.obsp[connectivity_key])
    dc = nx.algorithms.centrality.degree_centrality(g)
    adata.obs[f'{connectivity_key}_degree_centrality'] = list(dc.values())
    
    degrees = [d for (n,d) in g.degree()]
    adata.obs[f'{connectivity_key}_degree'] = degrees

In [3]:
def make_statannotations_pairs(i, j, mode='inner'):
    if mode == 'inner':
        pairs = []
        for x in i:
            pairs.extend(list(combinations(product([x],j), 2)))
            
    if mode == 'full':
         pairs = list(combinations(product(i,j),2))
            
    return pairs

In [4]:
candidates = pd.read_csv('./immunoprofile_hne_rag_features__passing_14_subset.csv', index_col=0)
candidates = candidates.drop('IP_19_E00218')

microhet_mapper = candidates['any_diff_edge']
microhet_mapper.name = 'microhet'
microhet_mapper = microhet_mapper.map({False:'Homog.', True:'Het.'})

tilewise = pd.read_csv('/mnt/disks/image_data/immunoprofile/ccrcc_subset_processing/inference_outputs/ccrcc_51_immunoprofile_passing_twostage_segmentation_tile_level_info.csv')
tilewise['case_id'] = tilewise['unique_id'].apply(lambda x: x.split('_HnE')[0])
tilewise = tilewise.set_index(['case_id'])

segmean = tilewise.groupby(['case_id','merged_labels']).mean()
segmean['gs'] = segmean['smoothed_prob_g4_not_g2']
segmean['ts'] = segmean['smoothed_prob_tumor']
segmean.index.names = ['case_id','seg_label']

#### Load annotated AnnData objects
- Includes various spatial neighbor setups, connectivity filters, and centrality calculations

In [5]:
out_dir = '/mnt/disks/image_data/immunoprofile/ccrcc_subset_processing/squidpy_adata_h5ad_files'
parse_template = os.path.join(out_dir, '{case_id}_all_area__cd8+_tumor+__bcell_filter___multistrat_radius={radius}__edge_anno_added.h5ad')
parses = format_parses(parse_template, glob(out_dir+'/*cd8+_tumor+__bcell_filter*__edge_anno_added*'))

regen_adata = {}
for idx, row in parses.iterrows():
    regen_adata[row['case_id']] = sc.read_h5ad(row['filepath'])

#### Incorporate Manual Review 
- IP_20_J00195: becomes homog.
- IP_19_G00660: loses one proximal edge but retains MH label

In [6]:
# case with Pancreatic tissue that got called as a heterogeneously distinct tumor region
microhet_mapper.loc['IP_20_J00195'] = 'Homog.'

In [7]:
# two regions that were adjacent tissue at met site
nodes_to_remove = {
    'IP_19_G00660': 1.0,
    'IP_20_J00195': 1.0,
}

In [8]:
for case_id, node_idx in nodes_to_remove.items():
    adata = regen_adata[case_id]
    crit = adata.obs['merged_labels'] == node_idx
    adata.obs.loc[crit, ['merged_labels','seg_label']] = 0  # equivalent columns; overwrite both
    print(f'Changed Node {node_idx} in {case_id} to be 0')

Changed Node 1.0 in IP_19_G00660 to be 0
Changed Node 1.0 in IP_20_J00195 to be 0


In [9]:
candidates.biopsySite.sort_values()

case_id
IP_20_M00245         Kidney
IP_19_G00562           Lung
IP_19_G00660           Lung
IP_19_J00132           Lung
IP_19_K00058           Lung
IP_19_T00810           Lung
IP_19_F00568       Lymphoid
IP_19_R00053       Lymphoid
IP_19_D00209           None
IP_19_K00367           None
IP_18_A00093           None
IP_20_J00195       Pancreas
IP_20_F00356    Soft Tissue
Name: biopsySite, dtype: object

### Optional: generate cell-edge DF

In [10]:
# edge_df_agg = pd.DataFrame()
# for case_id, adata in regen_adata.items():
#     print(case_id)
#     # reformat indices to be relative to DF position again 
#     adata.obs = adata.obs.reset_index(drop=True)
#     adata.obs.index= adata.obs.index.astype(str)

#     agg = fetch_full_edge_df(adata)
#     agg['case_id'] = case_id
    
#     # set up tumor-immune interaction labels
#     crit = (agg['dct_a'] == 'Tumor+ PDL1 Low') | (agg['dct_b'] == 'Tumor+ PDL1 Low')
#     agg.loc[crit, 'tumor_pdl1'] = 'PDL1 Low'

#     crit = (agg['dct_a'] == 'Tumor+ PDL1 High') | (agg['dct_b'] == 'Tumor+ PDL1 High')
#     agg.loc[crit, 'tumor_pdl1'] = 'PDL1 High'

#     crit = (agg['dct_a'] == 'CD8+ PD1 Low') | (agg['dct_b'] == 'CD8+ PD1 Low')
#     agg.loc[crit, 'cd8_pd1'] = 'PD1 Low'

#     crit = (agg['dct_a'] == 'CD8+ PD1 High') | (agg['dct_b'] == 'CD8+ PD1 High')
#     agg.loc[crit, 'cd8_pd1'] = 'PD1 High'

#     agg['edge_semantic_label'] = 'Tumor+ ' + agg['tumor_pdl1'] + ' vs CD8+ ' + agg['cd8_pd1']

#     # set up self-interaction labels wihtin subtypes
#     for dct in ['CD8+ PD1 High', 'CD8+ PD1 Low','Tumor+ PDL1 High','Tumor+ PDL1 Low']:
#         crit = (agg['dct_a'] == dct) & (agg['dct_b'] == dct)
#         agg.loc[crit, 'edge_semantic_label'] = f'{dct} [Self]'

#     # set up self interaction with diff submarker labels
#     crit = (agg['ct_a'] == 'CD8+') & (agg['ct_a'] == agg['ct_b']) & (agg['dct_a'] != agg['dct_b'])
#     agg.loc[crit ,'edge_semantic_label'] = 'CD8+ PD1 High/Low'

#     crit = (agg['ct_a'] == 'Tumor+') & (agg['ct_a'] == agg['ct_b']) & (agg['dct_a'] != agg['dct_b'])
#     agg.loc[crit ,'edge_semantic_label'] = 'Tumor+ PDL1 High/Low'
    
    
#     for edge_type, subdf in agg.groupby('edge_semantic_label'):
#         # print(edge_type)
#         adata.obs[edge_type] = 0
#         nodeset = set(subdf['a']).union(set(subdf['b']))
#         nodeset = [str(x) for x in nodeset]
#         adata.obs.loc[nodeset, edge_type] = 1
        
        
#     edge_df_agg = pd.concat([edge_df_agg, agg])
    


# edge_df_agg['edge_context'] = edge_df_agg['edge_semantic_label'].apply(lambda x: ('Tumor+' in x) & ('CD8+' in x))
# edge_df_agg['edge_context'] = edge_df_agg['edge_context'].map({False:'Self-Interaction', True:'Tumor-Immune Interaction'})

# for case_id, adata in regen_adata.items():
#     agg = edge_df_agg.set_index('case_id').loc[case_id]
#     for edge_type, subdf in agg.groupby('edge_context'):
#         # print(edge_type)
#         adata.obs[edge_type] = 0
#         nodeset = set(subdf['a']).union(set(subdf['b']))
#         nodeset = [str(x) for x in nodeset]
#         adata.obs.loc[nodeset, edge_type] = 1
        
#     adata.obs['edge_context'] = (adata.obs['Tumor-Immune Interaction'] == 1).map({True:'Tumor-Immune Interaction', False:'Self/Isolated'})

### Regenerate distance aggregation correctly while also creating cellwise DF for selected regions with minimum putative tumor content

#### Add PDL1-specific Edge Context

In [11]:
for case_id, adata in regen_adata.items():
    
    # add collapsed CD8 back in 
    adata.obs['collapsed_cd8'] = adata.obs['detailed_cell_type'].astype(str)
    crit = adata.obs['detailed_cell_type'].str.startswith('CD8+')
    adata.obs.loc[crit, 'collapsed_cd8'] = 'CD8+'
    adata.obs['collapsed_cd8'] = adata.obs['collapsed_cd8'].astype('category')

    # add collapsed Tumor back in 
    adata.obs['collapsed_tumor'] = adata.obs['detailed_cell_type'].astype(str)
    crit = adata.obs['detailed_cell_type'].str.startswith('Tumor+')
    adata.obs.loc[crit, 'collapsed_tumor'] = 'Tumor+'
    adata.obs['collapsed_tumor'] = adata.obs['collapsed_tumor'].astype('category')
    
    adata.obs['edge_context_pdl1_focus'] = adata.obs['edge_context'].astype(str)

    adata.obs['Tumor+ PDL1 High vs CD8+'] = adata.obs['Tumor+ PDL1 High vs CD8+ PD1 High'] + adata.obs['Tumor+ PDL1 High vs CD8+ PD1 Low']
    adata.obs['Tumor+ PDL1 Low vs CD8+'] = adata.obs['Tumor+ PDL1 Low vs CD8+ PD1 High'] + adata.obs['Tumor+ PDL1 Low vs CD8+ PD1 Low']

    # collapse cases where a CD8 contacts both PDL1 Low and High Tumor cells
    crit = (adata.obs['Tumor+ PDL1 High vs CD8+'] > 0)
    adata.obs.loc[crit, 'edge_context_pdl1_focus'] = 'Tumor-Immune Interaction (PDL1 High)'

    crit = (adata.obs['Tumor+ PDL1 High vs CD8+'] == 0) & (adata.obs['Tumor+ PDL1 Low vs CD8+'] > 0)
    adata.obs.loc[crit, 'edge_context_pdl1_focus'] = 'Tumor-Immune Interaction (PDL1 Low)'


In [12]:
adata.obs['CD8+ PD1 High [Self]']

0        0
1        0
2        0
3        0
4        0
        ..
53376    0
53377    0
53378    0
53379    0
53380    0
Name: CD8+ PD1 High [Self], Length: 53381, dtype: int64

In [13]:
list(filter(lambda x: x.startswith('CD8+'), adata.obs.columns))

['CD8+ PD1 High [Self]', 'CD8+ PD1 High/Low', 'CD8+ PD1 Low [Self]']

In [14]:
list(filter(lambda x: x.startswith('Tumor+'), adata.obs.columns))

['Tumor+ PDL1 High [Self]',
 'Tumor+ PDL1 High vs CD8+ PD1 High',
 'Tumor+ PDL1 High vs CD8+ PD1 Low',
 'Tumor+ PDL1 High/Low',
 'Tumor+ PDL1 Low [Self]',
 'Tumor+ PDL1 Low vs CD8+ PD1 High',
 'Tumor+ PDL1 Low vs CD8+ PD1 Low',
 'Tumor+ PDL1 High vs CD8+',
 'Tumor+ PDL1 Low vs CD8+']

In [15]:
detailed_interation_names = [
    'Tumor+ PDL1 High [Self]',
    'Tumor+ PDL1 Low [Self]',
    'Tumor+ PDL1 High/Low',
    'Tumor+ PDL1 High vs CD8+ PD1 High',
    'Tumor+ PDL1 High vs CD8+ PD1 Low',
    'Tumor+ PDL1 Low vs CD8+ PD1 High',
    'Tumor+ PDL1 Low vs CD8+ PD1 Low',
    'CD8+ PD1 High [Self]', 
    'CD8+ PD1 High/Low', 
    'CD8+ PD1 Low [Self]'
]

In [16]:
def plot_adata_neighbors(adata, connectivity_key, hues=['cell_type'], edges_width=1.5, image=None, s = 25, ax=None):
    sub = adata.copy()
    
    spatial_key = "spatial"
    library_id = "test_fov"
    sub.uns[spatial_key] = {library_id: {}}
    sub.uns[spatial_key][library_id]["images"] = {}
    sub.uns[spatial_key][library_id]["images"] = {"hires": image}
    sub.uns[spatial_key][library_id]["scalefactors"] = {"tissue_hires_scalef": 1, "spot_diameter_fullres":s}

    sq.pl.spatial_scatter(sub, ax=ax, color=hues, edges_width=edges_width, edges_color='black', outline_color=('black','black'), img_alpha=0.8, 
                          connectivity_key=connectivity_key, img=image, legend_fontsize='medium')

# Top-K PTs wrt T-I -- Ripley's L

In [17]:
ct_ec_groups = [
    'CD8+ : Tumor-Immune Interaction',
    'CD8+ : Self/Isolated',
    'Tumor+ : Tumor-Immune Interaction',
    'Tumor+ : Self/Isolated',
]


- so some cases have much lower cell density from the perspective of the pseudotiles

### Implement Test Statistics for Ripley's K 
- Diggle91
- Hahn12

In [18]:
from numpy.random import shuffle
from dask.distributed import Client

def calc_smoothed_variance_estimator(data, stat_col='k_stat', ):
    r_0 = data['bins'].max()
    
    var = data.groupby('bins')[stat_col].apply(lambda x: np.var(x, ddof=1))
    var.name = 'var'
    
    tmp = data.groupby('bins')[stat_col].apply(lambda x: np.var(x, ddof=1)).reset_index()
    ratios =  tmp[stat_col]/tmp['bins']**2
    const = ratios.sum()/r_0
    
    var_smooth = const * tmp['bins']**2
    
    tmp['smoothed_variance'] = var_smooth
    tmp = tmp.set_index('bins')['smoothed_variance']
    
    return tmp

def calculate_studentized_ripley_t(a,b, stat_col='k_stat', debug=False):
    """
    a: population a ripley statistics 
    b: population b ripley statistics 
    
    Follows "A Studentized Permutation Test for the Comparison of Spatial Point Patterns" (2012)
    """
    var_a = calc_smoothed_variance_estimator(a)/len(a)
    var_b = calc_smoothed_variance_estimator(b)/len(b)
    
    mu_a = a.groupby('bins')[stat_col].mean()
    mu_b = b.groupby('bins')[stat_col].mean()
    
    stat = (mu_a - mu_b)**2 / (var_a + var_b)
    
    if debug:
        return var_a, var_b, mu_a, mu_b
    else:
        return stat.dropna().sum()

def diggle91_dask(df, col, key_a, key_b, stat_col='k_stat'):
    dig = []
    for r in df['bins'].unique():
        a = df.loc[(df[col] == key_a) & (df['bins'] == r)]
        b = df.loc[(df[col] == key_b) & (df['bins'] == r)]

        numer = (a[stat_col].mean() - b[stat_col].mean())**2
        denom = (r**2 / a['num_cells'].sum()) + (r**2 / b['num_cells'].sum())
            
        dig.append(0.5 * numer / denom)
    return pd.Series(dig).dropna().sum()
        

### PT 2k, minimum EC 50; top-10; run all comparisons
- Do a separated Ripley calculation set for each individual `cluster_key` label

In [19]:
sc.settings.verbosity = 0

top_k = 10

pseudo_tile_size = 2000

n_neighbors = 8

min_cells = 50
print(f'setting min_cells to {min_cells}')

max_dist = 1500

# Ripley calc
mode = 'L'
n_observations = 1000
n_simulations = 0

max_ignore_frac = 0.5

ripley_store = pd.DataFrame()
ripley_singles = pd.DataFrame()


calc_distances = False
distance_key = 'unpruned_delaunay_distances'
distance_store = {}
distance_store_full = {}

subset_agg = pd.DataFrame()

counter = 0 

run_plotting = False

for case_id, adata in regen_adata.items():
    counter += 1 
    
    try:        
        print(case_id)
        df = adata.obs
        df['pseudo_x'] = df['cell_x_harmonized'] // pseudo_tile_size
        df['pseudo_y'] = df['cell_y_harmonized'] // pseudo_tile_size
        df['px'] = df['pseudo_x']
        df['py'] = df['pseudo_y']

        # general T-I context
        # Use collapsed tumor
        df['dct_ec'] = df['collapsed_tumor'].astype(str) + ' : ' + df['edge_context'].astype(str)
        df['dct_ec'] = df['dct_ec'].astype('category')

        df['ct_ec'] = df['cell_type'].astype(str) + ' : ' + df['edge_context'].astype(str)
        df['ct_ec'] = df['ct_ec'].astype('category')

        # PDL1 specific T-I context
        # use collapsed CD8 here 
        df['ct_ec_pdl1_specific'] = df['collapsed_cd8'].astype(str) + ' : ' + df['edge_context_pdl1_focus'].astype(str)
        df['ct_ec_pdl1_specific'] = df['ct_ec_pdl1_specific'].astype('category')
        
        # PDL1 specific T-I context
        # full detail
        df['dct_ec_pdl1_specific'] = df['detailed_cell_type'].astype(str) + ' : ' + df['edge_context_pdl1_focus'].astype(str)
        df['dct_ec_pdl1_specific'] = df['dct_ec_pdl1_specific'].astype('category')
        
        df['case_id'] = case_id

        pass_count = 0
        total = 0
        
        
        df_clone = df.copy()
        df_clone['omit'] = df_clone['seg_label'].isin([-1,0])

        df_clone = df_clone.loc[df_clone['edge_context'] =='Tumor-Immune Interaction']
        
        candidate_pts = get_indices(df_clone.groupby(['px','py'])['omit'].mean() <= max_ignore_frac)

        filtered_pts = []
        for (px, py), subdf in df_clone.set_index(['px','py']).loc[candidate_pts].groupby(['px','py']):
            counts = subdf['ct_ec'].astype(str).value_counts()
            if (len(counts) == 2) & (counts.min() > min_cells):
                filtered_pts.append((px,py))

        df_clone = df_clone.set_index(['px','py']).loc[filtered_pts]
        sorted_top_counts = df_clone.value_counts(['ct_ec','px','py']).loc['CD8+ : Tumor-Immune Interaction'].head(top_k)
        print(sorted_top_counts)
        subset_indices = sorted_top_counts.index.values
        df_clone = df_clone.loc[subset_indices]
        
        for (px,py), _ in tqdm_notebook(df_clone.groupby(['px','py'])):
            print((px,py))
            sub = adata[(adata.obs['pseudo_x'] == px) & (adata.obs['pseudo_y'] == py)]
            

            ignore_frac = sub.obs['seg_label'].isin([-1,0]).mean()

            if (ignore_frac <= max_ignore_frac):
                # ensure `min_cell` present for both CD8 and Tumor
                if len(sub.obsp['unpruned_delaunay_tumor_immune_connectivities'].nonzero()[0]) >= 0:
                    # use unfiltered df when storing here
                    subset_agg = pd.concat([subset_agg, sub.obs])
                    
                    # for cluster_key in ['ct_ec','ct_ec_pdl1_specific','dct_ec']:
                    for cluster_key in ['cell_type','ct_ec','ct_ec_pdl1_specific','dct_ec','dct_ec_pdl1_specific']:
                        total += 1

                        # filter out cells with < n_neighbors in that label
                        passing_ck_types = get_indices(sub.obs[cluster_key].value_counts() >= n_neighbors)
                        sub = sub[sub.obs[cluster_key].isin(passing_ck_types)]

                        if calc_distances:
                            for ck in ['unpruned_delaunay_self_CD8+_connectivities','unpruned_delaunay_self_Tumor+_connectivities','unpruned_delaunay_tumor_immune_connectivities',]:
                                if 'self' in ck:
                                    for context in ['Self/Isolated', 'Tumor-Immune Interaction']:
                                        for cell in ['Tumor+','CD8+']:
                                            subsub = sub[(sub.obs['cell_type'] == cell) & (sub.obs['edge_context'] == context)]
                                            a, b = subsub.obsp[ck].nonzero()
                                            distance_store[(case_id, px, py, ck, cell, context)] = subsub.obsp[distance_key][a,b].mean()
                                            distance_store_full[(case_id, px, py, ck, cell, context)] = subsub.obsp[distance_key][a,b]

                                else:
                                    a, b = sub.obsp[ck].nonzero()
                                    distance_store[(case_id, px, py, ck, 'Tumor+ & CD8+', 'Tumor-Immune Interaction')] = sub.obsp[distance_key][a,b].mean()
                                    distance_store_full[(case_id, px, py, ck, 'Tumor+ & CD8+', 'Tumor-Immune Interaction')] = sub.obsp[distance_key][a,b]


                        try:
                            # reset category to be relative to this pseudotile
                            sub.obs[cluster_key] = sub.obs[cluster_key].astype(str).astype('category')
                            for mode in ['L']:
                                sq.gr.ripley(sub, cluster_key=cluster_key, mode=mode, max_dist=max_dist, n_observations=n_observations, n_neigh=n_neighbors, n_simulations=n_simulations)
                                tmp = sub.uns[f'{cluster_key}_ripley_{mode}'][f'{mode}_stat']
                                tmp = tmp.rename(columns={cluster_key:'label'})
                                tmp['ripley_mode'] = mode
                                tmp['pseudo_x'] = px
                                tmp['pseudo_y'] = py
                                tmp['case_id'] = case_id
                                tmp['cluster_key'] = cluster_key
                                ripley_store = pd.concat([ripley_store, tmp])
                                
                                
                                # now do per-label individual Ripley calculations
                                for x in sub.obs[cluster_key].unique():
                                    sub_single = sub[sub.obs[cluster_key] == x]

                                    sq.gr.ripley(sub_single, cluster_key=cluster_key, mode=mode, max_dist=max_dist, n_observations=n_observations, n_neigh=n_neighbors, n_simulations=n_simulations)
                                    tmp = sub_single.uns[f'{cluster_key}_ripley_{mode}'][f'{mode}_stat']
                                    tmp = tmp.rename(columns={cluster_key:'label'})
                                    tmp['label'] = x
                                    tmp['ripley_mode'] = mode
                                    tmp['pseudo_x'] = px
                                    tmp['pseudo_y'] = py
                                    tmp['case_id'] = case_id
                                    tmp['cluster_key'] = cluster_key
                                    ripley_singles = pd.concat([ripley_singles,tmp])
    
#                                 if run_plotting:
#                                     print(sub.obs.value_counts([cluster_key], sort=False))
#                                     set_rc(6,6)
#                                     sq.pl.ripley(sub, cluster_key=cluster_key, mode=mode,)
#                                     plt.show()
                                
                            pass_count += 1
                            set_rc(12,12, palette='colorblind', font_scale=1.6)
                            # regenerate sub adata fresh for plotting (since we omit sparse cell types prior to Ripley calc)
                            sub_regen = adata[(adata.obs['pseudo_x'] == px) & (adata.obs['pseudo_y'] == py)]
                    


                            if run_plotting:
                                ax = plt.gca()
                                plot_adata_neighbors(sub_regen, 'unpruned_delaunay_tumor_immune_connectivities', ['ct_ec'], s=15, edges_width=0.25, ax=ax)
                                plt.show()
                            
                        except Exception as e:
                            print(f'Error with {case_id}, {px}, {py}:    {e}')
                            print(e)
                            pass
                else:
                    pass
                    # print(f'Omitting {px}, {py} with {len(sub.obs)} cells')
                    # print(sub.obs[cluster_key].value_counts())

        print(f'Successfully grabbed {pass_count}/{total} PT [within putative tumor seg regions]')
    except Exception as e:
        print(f'Could not process {case_id}; \n {e}')
    
#     if counter > 0:
#         break
    
ripley_store = ripley_store.set_index('case_id').join(microhet_mapper).reset_index()
ripley_singles = ripley_singles.set_index('case_id').join(microhet_mapper).reset_index()

for col in ['cell_type','ct_ec','ct_ec_pdl1_specific','dct_ec_pdl1_specific', 'dct_ec']:
    subset_agg[col] = subset_agg[col].astype('category')

subset_agg = subset_agg.set_index('case_id').join(microhet_mapper).reset_index()


setting min_cells to 50
IP_19_K00367
px    py  
7.0   19.0    1971
6.0   18.0    1933
3.0   9.0     1927
12.0  27.0    1830
4.0   19.0    1774
6.0   19.0    1761
4.0   18.0    1760
8.0   6.0     1709
12.0  28.0    1699
3.0   8.0     1686
dtype: int64


  0%|          | 0/10 [00:00<?, ?it/s]

(3.0, 8.0)
(3.0, 9.0)
(4.0, 18.0)
(4.0, 19.0)
(6.0, 18.0)
(6.0, 19.0)
(7.0, 19.0)
(8.0, 6.0)
(12.0, 27.0)
(12.0, 28.0)
Successfully grabbed 50/50 PT [within putative tumor seg regions]
IP_19_D00209
px    py  
18.0  15.0    214
19.0  21.0    157
15.0  11.0    153
4.0   21.0    146
5.0   5.0     140
18.0  23.0    136
4.0   22.0    135
18.0  22.0    130
      21.0    128
17.0  21.0    126
dtype: int64


  0%|          | 0/10 [00:00<?, ?it/s]

(4.0, 21.0)
(4.0, 22.0)
(5.0, 5.0)
(15.0, 11.0)
(17.0, 21.0)
(18.0, 15.0)
(18.0, 21.0)
(18.0, 22.0)
(18.0, 23.0)
(19.0, 21.0)
Successfully grabbed 50/50 PT [within putative tumor seg regions]
IP_19_J00132
px    py 
9.0   8.0    194
8.0   7.0    128
10.0  7.0    118
9.0   7.0    112
12.0  4.0     99
10.0  9.0     98
9.0   5.0     93
10.0  8.0     91
11.0  7.0     88
13.0  8.0     82
dtype: int64


...

(14.0, 14.0)
(15.0, 12.0)
(16.0, 5.0)
(16.0, 12.0)
(16.0, 18.0)
(17.0, 5.0)
(17.0, 8.0)
(17.0, 11.0)
(17.0, 12.0)
(17.0, 15.0)
Successfully grabbed 50/50 PT [within putative tumor seg regions]
IP_18_A00093
px    py 
16.0  5.0    142
17.0  7.0    142
15.0  4.0    114
18.0  7.0    111
17.0  6.0    110
13.0  7.0    102
17.0  4.0     99
12.0  6.0     98
15.0  7.0     97
16.0  7.0     94
dtype: int64


  0%|          | 0/10 [00:00<?, ?it/s]

(12.0, 6.0)
(13.0, 7.0)
(15.0, 4.0)
(15.0, 7.0)
(16.0, 5.0)
(16.0, 7.0)
(17.0, 4.0)
(17.0, 6.0)
(17.0, 7.0)
(18.0, 7.0)
Successfully grabbed 50/50 PT [within putative tumor seg regions]
IP_20_J00195
Could not process IP_20_J00195; 
 'CD8+ : Tumor-Immune Interaction'
IP_19_K00058
px    py  
8.0   12.0    392
      13.0    389
7.0   12.0    377
8.0   15.0    369
9.0   13.0    358
      12.0    313
8.0   16.0    257
9.0   14.0    230
6.0   16.0    222
10.0  12.0    220
dtype: int64


  0%|          | 0/10 [00:00<?, ?it/s]

(6.0, 16.0)
(7.0, 12.0)
(8.0, 12.0)
(8.0, 13.0)
(8.0, 15.0)
(8.0, 16.0)
(9.0, 12.0)
(9.0, 13.0)
(9.0, 14.0)
(10.0, 12.0)
Successfully grabbed 50/50 PT [within putative tumor seg regions]
IP_19_G00562
Could not process IP_19_G00562; 
 'CD8+ : Tumor-Immune Interaction'


In [ ]:
subset_agg.to_csv('./immune_hotspot_topk_cellwise_data__subset_agg.csv')
ripley_store.to_csv('./immune_hotspot_topk_ripley_agg_data__subset_agg.csv')
ripley_singles.to_csv('./immune_hotspot_topk_ripley_singles_data__subset_agg.csv')